In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"
global plz_work

In [2]:
# encoding: utf-8

"""
Stdout/stderr redirector, at the OS level, using file descriptors.

This also works under windows.
"""

__docformat__ = "restructuredtext en"

#-------------------------------------------------------------------------------
#  Copyright (C) 2008  The IPython Development Team
#
#  Distributed under the terms of the BSD License.  The full license is in
#  the file COPYING, distributed as part of this software.
#-------------------------------------------------------------------------------


import os
import sys

STDOUT = 1
STDERR = 2

class FDRedirector(object):
    """ Class to redirect output (stdout or stderr) at the OS level using
        file descriptors.
    """ 

    def __init__(self, fd=STDOUT):
        """ fd is the file descriptor of the outpout you want to capture.
            It can be STDOUT or STERR.
        """
        self.fd = fd
        self.started = False
        self.piper = None
        self.pipew = None

    def start(self):
        """ Setup the redirection.
        """
        if not self.started:
            self.oldhandle = os.dup(self.fd)
            self.piper, self.pipew = os.pipe()
            os.dup2(self.pipew, self.fd)
            os.close(self.pipew)

            self.started = True

    def flush(self):
        """ Flush the captured output, similar to the flush method of any
        stream.
        """
        if self.fd == STDOUT:
            sys.stdout.flush()
        elif self.fd == STDERR:
            sys.stderr.flush()

    def stop(self):
        """ Unset the redirection and return the captured output. 
        """
        if self.started:
            self.flush()
            os.dup2(self.oldhandle, self.fd)
            os.close(self.oldhandle)
            f = os.fdopen(self.piper, 'r')
            output = f.read()
            f.close()

            self.started = False
            return output
        else:
            return ''

    def getvalue(self):
        """ Return the output captured since the last getvalue, or the
        start of the redirection.
        """
        output = self.stop()
        self.start()
        return output

In [3]:
# TAKEN FROM https://github.com/adler-j/adler

import demandimport
with demandimport.enabled():
    import tensorflow as tf

    
__all__ = ('image_grid', 'image_grid_summary', 'scalars_summary')


def image_grid(x, size=8):
    t = tf.unstack(x[:size * size], num=size*size, axis=0)
    rows = [tf.concat(t[i*size:(i+1)*size], axis=0) for i in range(size)]
    image = tf.concat(rows, axis=1)
    return image[None]


def image_grid_summary(name, x):
    with tf.name_scope(name):
        tf.summary.image('grid', image_grid(x))


def scalars_summary(name, x):
    with tf.name_scope(name):
        x = tf.reshape(x, [-1])
        mean, var = tf.nn.moments(x, axes=0)
        tf.summary.scalar('mean', mean)
        tf.summary.scalar('std', tf.sqrt(var))
        tf.summary.histogram('histogram', x)
        
import os
import shutil
from os.path import join, expanduser, exists

import demandimport
with demandimport.enabled():
    import tensorflow as tf

__all__ = ('get_base_dir',
           'default_checkpoint_path', 'default_tensorboard_dir',
           'summary_writers')


def get_base_dir():
    """Get the data directory."""
    base_odl_dir = os.environ.get('ADLER_HOME',
                                  expanduser(join('~', '.adler')))
    data_home = join(base_odl_dir, 'tensorflow')
    if not exists(data_home):
        os.makedirs(data_home)
    return data_home


def default_checkpoint_path(name):
    checkpoint_dir = join(get_base_dir(), 'checkpoints')
    if not exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    checkpoint_path = join(checkpoint_dir,
                           '{}.ckpt'.format(name))

    return checkpoint_path


def default_tensorboard_dir(name):
    tensorboard_dir = join(get_base_dir(), 'tensorboard', name)
    if not exists(tensorboard_dir):
        os.makedirs(tensorboard_dir)
    return tensorboard_dir


def summary_writers(name, cleanup=False, session=None, write_graph=True):
    if session is None:
        session = tf.get_default_session()

    dname = default_tensorboard_dir(name)

    if cleanup and os.path.exists(dname):
        shutil.rmtree(dname, ignore_errors=True)

    if write_graph:
        graph = session.graph
    else:
        graph = None

    test_summary_writer = tf.summary.FileWriter(dname + '/test', graph)
    train_summary_writer = tf.summary.FileWriter(dname + '/train')

    return test_summary_writer, train_summary_writer


import demandimport
with demandimport.enabled():
    import tensorflow as tf

import numpy as np

def cosine_decay(learning_rate, global_step, maximum_steps,
                 name=None):
    from tensorflow.python.ops import math_ops
    from tensorflow.python.framework import ops

    if global_step is None:
        raise ValueError("global_step is required for cosine_decay.")
    with ops.name_scope(name, "CosineDecay",
                      [learning_rate, global_step, maximum_steps]) as name:
        learning_rate = ops.convert_to_tensor(learning_rate, name="learning_rate")
        dtype = learning_rate.dtype
        global_step = math_ops.cast(global_step, dtype)
        maximum_steps = math_ops.cast(maximum_steps, dtype)

        p = tf.mod(global_step / maximum_steps, 1)

    return learning_rate * (0.5 + 0.5 * math_ops.cos(p * np.pi))


class EMAHelper(object):
    def __init__(self, decay=0.99, session=None):
        if session is None:
            self.session = tf.get_default_session()
        else:
            self.session = session

        self.all_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        self.ema = tf.train.ExponentialMovingAverage(decay=decay)
        self.apply = self.ema.apply(self.all_vars)
        self.averages = [self.ema.average(var) for var in self.all_vars]

    def average_dict(self):
        ema_averages_results = self.session.run(self.averages)
        return {var: value for var, value in zip(self.all_vars,
                                                 ema_averages_results)}

    def variables_to_restore(self):
        return self.ema.variables_to_restore(tf.moving_average_variables())


def ema_wrapper(is_training, decay=0.99, scope='ema_wrapper', reuse=False):
    """Use Exponential Moving Average of weights during testing.

    Parameters
    ----------
    is_training : bool or `tf.Tensor` of type bool
        Indicates if the EMA should be applied or not
    decay:

    Examples
    --------
    During training, the current value of a is used. During testing, the
    exponential moving average is applied instead.

    >>> @ema_wrapper(is_training)
    ... def function(x):
    ....    a = tf.get_variable('a', [], tf.float32)
    ...     return a * x
    """
    def function(fun):
        def fun_wrapper(*args, **kwargs):
            with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
                # Regular call
                with tf.variable_scope('function_call') as sc:
                    result_train = fun(*args, **kwargs)

                # Set up exponential moving average
                ema = tf.train.ExponentialMovingAverage(decay=decay)
                var_class = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                              sc.name)
                ema_op = ema.apply(var_class)

                # Add to collection so they are updated
                tf.add_to_collection(tf.GraphKeys.UPDATE_OPS, ema_op)

                # Getter for the variables with EMA applied
                def ema_getter(getter, name, *args, **kwargs):
                    var = getter(name, *args, **kwargs)
                    ema_var = ema.average(var)
                    return ema_var if ema_var else var

                # Call with EMA applied
                with tf.variable_scope('function_call',
                                       reuse=True,
                                       custom_getter=ema_getter):
                    result_test = fun(*args, **kwargs)

                # Return the correct version depending on if we're training or
                # not
                return tf.cond(is_training,
                               lambda: result_train, lambda: result_test)
        return fun_wrapper
    return function

In [4]:
"""Utilities for computing the sobolev norm."""

import tensorflow as tf

def sobolev_filter(x, c=5, s=1):
    """Apply sobolev filter.

    Parameters
    ----------
    x : tensorflow.Tensor of shape B W H C
        txt
    c : float
        Scaling of the cooridinate systems (1 / pixel size)
    s : float
        Order of the Sobolev norm
    """
    with tf.name_scope('sobolev'):
        # FFT is taken over the innermost axes, so move channel to beginning.
        x = tf.transpose(x, [0, 3, 1, 2])
        fft_x = tf.spectral.fft2d(tf.cast(x, 'complex64'))

        shape = tf.shape(fft_x)
        sx = shape[3]
        sy = shape[2]

        # Construct meshgrid for the scale
        x = tf.range(sx)
        x = tf.minimum(x, sx - x)
        x = tf.cast(x, dtype='complex64') / tf.cast(sx // 2, dtype='complex64')
        y = tf.range(sy)
        y = tf.minimum(y, sy - y)
        y = tf.cast(y, dtype='complex64') / tf.cast(sy // 2, dtype='complex64')
        X, Y = tf.meshgrid(x, y)
        X = X[None, None]
        Y = Y[None, None]

        scale = (1 + c * (X ** 2 + Y ** 2)) ** (s / 2)

        # Compute spatial gradient in fourier space
        fft_x = scale * fft_x

        result_x = tf.spectral.ifft2d(fft_x)
        result_x = tf.real(result_x)
        return tf.transpose(result_x, [0, 2, 3, 1])


In [5]:
# np.set_printoptions(threshold=np.nan)

def print_param(v):
    print("Name: ", v.name)
    vector = sess.run(v)
    print("Shape: ", v.shape)
    print(vector)
    print("=" * 20)
    name = v.name.replace("/", "_")
    np.save("params/" + name, vector)
    
def print_all():
    [print_param(v) for v in tf.trainable_variables() ]


In [6]:
# Source: https://gist.github.com/vihari/f9b361058825e16d390f0e443bfdffc7
# global vec0
vec0 = []
vec1 = []
vec2 = []

def tf_print(index, op, tensors, message=None):


    def print_message(x):
        if index == 0:
            global vec0
            vec0.append(x)
        elif index == 1:
            global vec1
            vec1.append(x)
        elif index == 2:
            global vec2
            vec2.append(x)

        sys.stdout.write(message + " %s\n" % x[0,:2,:2,:2])
        return x

    prints = [tf.py_func(print_message, [tensor], tensor.dtype) for tensor in tensors]
    with tf.control_dependencies(prints):
        op = tf.identity(op)
    return op

In [7]:
# Taken from https://github.com/adler-j/bwgan
"""Code for training Banach Wasserstein GAN on CIFAR 10.

With all the dependencies installed, the code should run as-is. 
Data is downloaded on the fly.
"""

import sys
import tensorflow as tf
import numpy as np
import tensordata
import functools

# User selectable parameters
EXPONENT = 2
SOBOLEV_C = 5.0
SOBOLEV_S = 0
MAX_ITERS = 100000
SUMMARY_FREQ = 10
INCEPTION_FREQ = 1000
BATCH_SIZE = 64
BATCH_SIZE_TEST = 100
reset = True

# set seeds for reproducibility
np.random.seed(0)
tf.set_random_seed(0)

sess = tf.InteractiveSession()

# Training specific parameters
name = 'cifar10_sobolev_5/p={}s={}'.format(EXPONENT, SOBOLEV_S)
size = 32
DUAL_EXPONENT = 1 / (1 - 1/EXPONENT) if EXPONENT != 1 else np.inf


with tf.name_scope('placeholders'):
#     x_train_ph, _ = tensordata.get_cifar10_tf(batch_size=BATCH_SIZE)
    x_test_ph, _ = tensordata.get_cifar10_tf(batch_size=BATCH_SIZE_TEST)

    is_training = tf.placeholder(bool, name='is_training')
    use_agumentation = tf.identity(is_training, name='is_training')


with tf.name_scope('pre_process'):
#     x_train = (x_train_ph - 0.5) * 2.0
    x_test = (x_test_ph - 0.5) * 2.0
    x_true = x_test

#     x_true = tf.cond(is_training,
#                      lambda: x_train,
#                      lambda: x_test)

def apply_conv(x, filters=32, kernel_size=3, he_init=True, print_time=False):
    if he_init:
        initializer = tf.contrib.layers.variance_scaling_initializer(uniform=True)
    else:
        initializer = tf.contrib.layers.xavier_initializer(uniform=True)

    return tf.layers.conv2d(x, filters=filters, kernel_size=kernel_size,
                            padding='SAME', kernel_initializer=initializer)


def activation(x):
    with tf.name_scope('activation'):
        return tf.nn.relu(x)


def bn(x, print_time=False):
    if print_time:
        before_bn = tf_print(0, x, [x], "BN INPUT: I FIXED THIS ALREADY")
        result= tf.layers.batch_normalization(before_bn,
                                        momentum=0.9, # OK
                                        center=True, # OK
                                        scale=True, #OK
                                        epsilon=1e-5,#OK
    #                                     zero_debias_moving_mean=False,
                                        training=True)
        return tf_print(1, result, [result], "BN RESULT: THIS SHOULD BE WORKING!!!")
    else:
        return tf.layers.batch_normalization(x,
                                        momentum=0.9, # OK
                                        center=True, # OK
                                        scale=True, #OK
                                        epsilon=1e-5,#OK
    #                                     zero_debias_moving_mean=False,
                                        training=True)



def stable_norm(x, ord):
    x = tf.contrib.layers.flatten(x)
    alpha = tf.reduce_max(tf.abs(x) + 1e-5, axis=1)
    result = alpha * tf.norm(x / alpha[:, None], ord=ord, axis=1)
    return result


def downsample(x):
    with tf.name_scope('downsample'):
        x = tf.identity(x)
        return tf.add_n([x[:,::2,::2,:], x[:,1::2,::2,:],
                         x[:,::2,1::2,:], x[:,1::2,1::2,:]]) / 4.

def upsample(x, print_time=False):
    with tf.name_scope('upsample'):
        if print_time:
            x_print1 = tf_print(3, x, [x], message="UPSAMPLING")
            x = tf.identity(x_print1)
            x = tf.concat([x, x, x, x], axis=-1)
            x_print2 = tf_print(4, x, [x], message="Weird reshape")

            upsampled = tf.depth_to_space(x_print2, 2)
            return tf_print(2, upsampled, [upsampled], message="Upsampled")
        else:
            x = tf.identity(x)
            x = tf.concat([x, x, x, x], axis=-1)
            upsampled = tf.depth_to_space(x, 2)
            return upsampled


def conv_meanpool(x, print_time=False, **kwargs):
    return downsample(apply_conv(x, **kwargs))

def meanpool_conv(x, print_time=False, **kwargs):
    return apply_conv(downsample(x), **kwargs)

def upsample_conv(x, print_time=False, **kwargs):
    return apply_conv(upsample(x, print_time=print_time), **kwargs)

def identity(x, print_time=False):
    return tf.identity(x)

def resblock(x, filters, resample=None, normalize=False, print_time=False):
    if normalize:
        norm_fn = bn
    else:
        norm_fn = identity

    if resample == 'down':
        conv_1 = functools.partial(apply_conv, filters=filters)
        conv_2 = functools.partial(conv_meanpool, filters=filters)
        conv_shortcut = functools.partial(conv_meanpool, filters=filters,
                                          kernel_size=1, he_init=False)
    elif resample == 'up':
        conv_1 = functools.partial(upsample_conv, filters=filters)
        conv_2 = functools.partial(apply_conv, filters=filters)
        conv_shortcut = functools.partial(upsample_conv, filters=filters,
                                          kernel_size=1, he_init=False)
    elif resample == None:
        conv_1 = functools.partial(apply_conv, filters=filters)
        conv_2 = functools.partial(apply_conv, filters=filters)
        conv_shortcut = tf.identity

    with tf.name_scope('resblock'):
        x = tf.identity(x)
        normed = norm_fn(x)
        activated = activation(normed)
        activation_printed = tf_print(7, activated, [activated], message="Activated")
        update = conv_1(activation_printed, print_time=True)
        update_print = tf_print(7, update, [update], message="AFTER conv1")
        normed_again = norm_fn(update_print, print_time=print_time)
        normed_again_print = tf_print(7, normed_again, [normed_again], message="Norm again")
        activated_again = activation(normed_again_print)
        activated_again_print = tf_print(7, activated_again, [activated_again], message="activated again")
        update = conv_2(activated_again_print)
        update_print2 = tf_print(7, update, [update], message="AFTER conv2")
        skip = conv_shortcut(x)
        return skip + update_print2


def resblock_optimized(x, filters):
    with tf.name_scope('resblock'):
        x = tf.identity(x)
        update = apply_conv(x, filters=filters)
        update = conv_meanpool(activation(update), filters=filters)

        skip = meanpool_conv(x, filters=128, kernel_size=1, he_init=False)
        return skip + update


def generator(z, reuse):
    with tf.variable_scope('generator', reuse=reuse):
        with tf.name_scope('pre_process'):
            z = tf.layers.dense(z, 4 * 4 * 128)
            x = tf.reshape(z, [-1, 4, 4, 128])

        with tf.name_scope('x1'):
            x = resblock(x, filters=128, resample='up', normalize=True, print_time=True) # 8
            x = resblock(x, filters=128, resample='up', normalize=True) # 16
            x = resblock(x, filters=128, resample='up', normalize=True) # 32

        with tf.name_scope('post_process'):
            x = activation(bn(x))
            result = apply_conv(x, filters=3, he_init=False)
            return tf.tanh(result)



def discriminator(x, reuse):
    with tf.variable_scope('discriminator', reuse=reuse):
        with tf.name_scope('pre_process'):
            x = resblock_optimized(x, filters=128)

        with tf.name_scope('x1'):
            x = resblock(x, filters=128, resample='down') # 8
            x = resblock(x, filters=128) # 16
            x = resblock(x, filters=128) # 32

        with tf.name_scope('post_process'):
            x = activation(x)
            x = tf.reduce_mean(x, axis=[1, 2])
            flat = tf.contrib.layers.flatten(x)
            flat = tf.layers.dense(flat, 1)
            return flat


with tf.name_scope('gan'):
    z = tf.random_normal([tf.shape(x_true)[0], 128], name="z")

    x_generated = generator(z, reuse=False)

    d_true = discriminator(x_true, reuse=False)
    d_generated = discriminator(x_generated, reuse=True)

    z_gen = tf.random_normal([BATCH_SIZE * 2, 128], name="z")
    d_generated_train = discriminator(generator(z_gen, reuse=True), reuse=True)

# with tf.name_scope('dual_norm'):
#     sobolev_true = sobolev_filter(x_true, c=SOBOLEV_C, s=SOBOLEV_S)
#     lamb = tf.reduce_mean(stable_norm(sobolev_true, ord=EXPONENT))
#     dual_sobolev_true = sobolev_filter(x_true, c=SOBOLEV_C, s=-SOBOLEV_S)
#     gamma = tf.reduce_mean(stable_norm(sobolev_true, ord=DUAL_EXPONENT))

# with tf.name_scope('regularizer'):
#     epsilon = tf.random_uniform([tf.shape(x_true)[0], 1, 1, 1], 0.0, 1.0)
#     x_hat = epsilon * x_generated + (1 - epsilon) * x_true
#     d_hat = discriminator(x_hat, reuse=True)

#     gradients = tf.gradients(d_hat, x_hat)[0]
#     dual_sobolev_gradients = sobolev_filter(gradients, c=SOBOLEV_C, s=-SOBOLEV_S)
#     ddx = stable_norm(dual_sobolev_gradients, ord=DUAL_EXPONENT)

#     d_regularizer = tf.reduce_mean(tf.square(ddx / gamma - 1))
#     d_regularizer_mean = tf.reduce_mean(tf.square(d_true))

# with tf.name_scope('loss_gan'):
#     wasserstein_scaled = (tf.reduce_mean(d_generated) - tf.reduce_mean(d_true))
#     wasserstein = wasserstein_scaled / gamma

#     g_loss = tf.reduce_mean(d_generated_train) / gamma
#     d_loss = (-wasserstein +
#               lamb * d_regularizer +
#               1e-5 * d_regularizer_mean)

# with tf.name_scope('optimizer'):
#     ema = EMAHelper(decay=0.99)

#     global_step = tf.Variable(0, trainable=False, name='global_step')
#     decay = tf.maximum(0., 1.-(tf.cast(global_step, tf.float32)/MAX_ITERS))
#     learning_rate = 2e-4 * decay
#     optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0., beta2=0.9)

#     update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope='gan/generator')
#     g_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')
#     with tf.control_dependencies(update_ops):
#         g_train = optimizer.minimize(g_loss, var_list=g_vars,
#                                      global_step=global_step)

#     update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope='gan/discriminator')
#     d_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
#     with tf.control_dependencies(update_ops):
#         d_train = optimizer.minimize(d_loss, var_list=d_vars)


# with tf.name_scope('summaries'):
#     tf.summary.scalar('wasserstein_scaled', wasserstein_scaled)
#     tf.summary.scalar('wasserstein', wasserstein)

#     tf.summary.scalar('g_loss', g_loss)

#     tf.summary.scalar('d_loss', d_loss)
#     scalars_summary('d_true', d_true)
#     scalars_summary('d_generated', d_generated)
#     tf.summary.scalar('d_regularizer', d_regularizer)
#     tf.summary.scalar('d_regularizer_mean', d_regularizer_mean)

#     tf.summary.scalar('learning_rate', learning_rate)
#     tf.summary.scalar('global_step', global_step)

#     scalars_summary('x_generated', x_generated)
#     scalars_summary('x_true', x_true)

#     scalars_summary('gamma', gamma)
#     scalars_summary('lamb', lamb)

#     image_grid_summary('x_true', x_true)
#     image_grid_summary('x_generated', x_generated)
#     image_grid_summary('gradients', gradients)
#     image_grid_summary('dual_sobolev_gradients', dual_sobolev_gradients)

#     scalars_summary('ddx', ddx)
#     scalars_summary('gradients', gradients)
#     scalars_summary('dual_sobolev_gradients', dual_sobolev_gradients)

#     merged_summary = tf.summary.merge_all()

#     # Advanced metrics
#     with tf.name_scope('inception'):
#         # Specific function to compute inception score for very large
#         # number of samples
#         def generate_and_classify(z):
#             INCEPTION_OUTPUT = 'logits:0'
#             x = generator(z, reuse=True)
#             x = tf.image.resize_bilinear(x, [299, 299])
#             return tf.contrib.gan.eval.run_inception(x, output_tensor=INCEPTION_OUTPUT)

#         # Fixed z for fairness between runs
#         inception_z = tf.constant(np.random.randn(10000, 128), dtype='float32')
#         inception_score = tf.contrib.gan.eval.classifier_score(inception_z,
#                                                                classifier_fn=generate_and_classify,
#                                                                num_batches=10000 // 100)

#         inception_summary = tf.summary.merge([
#                 tf.summary.scalar('inception_score', inception_score)])

#         full_summary = tf.summary.merge([merged_summary, inception_summary])

#     test_summary_writer, train_summary_writer = summary_writers(name, cleanup=reset, write_graph=False)
#
# Initialize all TF variables
sess.run([tf.global_variables_initializer(),
          tf.local_variables_initializer()])




print_all()

# assert False

# # Coordinate the loading of image files.
# coord = tf.train.Coordinator()
# threads = tf.train.start_queue_runners(coord=coord)

# # Add op to save and restore
# saver = tf.train.Saver()
# if not reset:
#     saver.restore(sess, default_checkpoint_path(name))

# # Standardized validation z
# z_validate = np.random.randn(BATCH_SIZE_TEST, 128)

# # Train the network
# while True:
#     i = sess.run(global_step)
#     if i >= MAX_ITERS:
#         break

#     num_d_train = 5
#     for j in range(num_d_train):
#         _, d_loss_result = sess.run([d_train, d_loss],
#                                     feed_dict={is_training: True})

#     _, g_loss_result, _ = sess.run([g_train, g_loss, ema.apply],
#              feed_dict={is_training: True})

#     print('s={}, i={}, j={}, d_loss={}, g_loss={}'.format(SOBOLEV_S, i, j,
#                                                     d_loss_result,
#                                                     g_loss_result))

#     if i % SUMMARY_FREQ == SUMMARY_FREQ - 1:
#         ema_dict = ema.average_dict()
#         merged_summary_result_train = sess.run(merged_summary,
#                                          feed_dict={is_training: False,
#                                                     **ema_dict})
#         train_summary_writer.add_summary(merged_summary_result_train, i)
#     if i % INCEPTION_FREQ == INCEPTION_FREQ - 1:
#         ema_dict = ema.average_dict()
#         merged_summary_result_test = sess.run(full_summary,
#                                          feed_dict={z: z_validate,
#                                                     is_training: False,
#                                                     **ema_dict})
#         test_summary_writer.add_summary(merged_summary_result_test, i)


#     if i % 1000 == 999:
#         saver.save(sess,default_checkpoint_path(name))

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To 

Shape:  (128,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Name:  generator/conv2d_2/kernel:0
Shape:  (1, 1, 128, 128)
[[[[ 0.01609883 -0.04642101 -0.00843991 ...  0.10992827  0.10723139
    -0.06542592]
   [-0.08368488  0.09893672 -0.0625662  ... -0.09366006 -0.08983783
     0.14140318]
   [-0.08029065  0.11328249 -0.01671912 ...  0.05396345 -0.13329582
    -0.02999389]
   ...
   [-0.0104098   0.047791   -0.0972296  ...  0.12787817 -0.14265963
    -0.07675136]
   [ 0.12741531 -0.03997076  0.11145    ...  0.04992394 -0.14526579
    -0.0339964 ]
   [-0.1270833   0.14487655 -0.05556089 ...  0.00691946  0.02808146
     0.02171192]]]]
Name:  generat

Name:  generator/conv2d_8/bias:0
Shape:  (128,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Name:  generator/batch_normalization_6/gamma:0
Shape:  (128,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
Name:  generator/batch_normalization_6/beta:0
Shape:  (128,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

Name:  discriminator/conv2d_6/bias:0
Shape:  (128,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Name:  discriminator/conv2d_7/kernel:0
Shape:  (3, 3, 128, 128)
[[[[ 0.04223923  0.02190776  0.00625471 ...  0.00765704 -0.04309656
     0.02649561]
   [ 0.06237982  0.06698683 -0.04274312 ...  0.04709339  0.07180689
    -0.00042572]
   [ 0.07191478 -0.02540761 -0.00383912 ...  0.03300861  0.02460765
    -0.01819198]
   ...
   [-0.06374326  0.00258094  0.06497268 ...  0.04500865 -0.00214269
     0.06460187]
   [ 0.03256065  0.03320195 -0.01791007 ... -0.06897612 -0.04783835
    -0.06056623]
   [ 0.05952722 -0.02287774  0.05288288 ... -0.02223444  0.02

In [8]:
def test_generator(input_arr):
    STDERR = 2
    redirect = FDRedirector(STDERR)
    redirect.start();
    tf_arr = tf.convert_to_tensor(input_arr, dtype=tf.float32)
    gen_output = generator(tf_arr, reuse=True)
    output = sess.run(gen_output, feed_dict={is_training: True})
    print_statements = redirect.stop()
    print("Print Statements")
    print(print_statements)
    print(output.shape)
    print("FINAL OUTPUT")
    print(output)
    first_dense_weight = sess.run(tf.trainable_variables()[0])
    sums = np.sum(first_dense_weight, axis=0)
    print("First Dense should be Should be", sums)
    

In [9]:
x = np.ones((1, 128))
test_generator(x)
raise ValueError("hi")

Activated [[[0.         0.21231355]
  [0.22819537 0.81528354]]

 [[0.         0.        ]
  [2.3898323  0.15784769]]]
UPSAMPLING [[[0.         0.21231355]
  [0.22819537 0.81528354]]

 [[0.         0.        ]
  [2.3898323  0.15784769]]]
Weird reshape [[[0.         0.21231355]
  [0.22819537 0.81528354]]

 [[0.         0.        ]
  [2.3898323  0.15784769]]]
Upsampled [[[0.         0.21231355]
  [0.         0.21231355]]

 [[0.         0.21231355]
  [0.         0.21231355]]]
AFTER conv1 [[[ 0.65031743 -0.6203244 ]
  [-0.46407175 -0.29227483]]

 [[ 0.3085447  -0.1323849 ]
  [-0.46749058 -0.29016238]]]
BN INPUT: I FIXED THIS ALREADY [[[ 0.65031743 -0.6203244 ]
  [-0.46407175 -0.29227483]]

 [[ 0.3085447  -0.1323849 ]
  [-0.46749058 -0.29016238]]]
BN RESULT: THIS SHOULD BE WORKING!!! [[[ 1.6407784 -1.2557573]
  [-0.4508798 -0.8140216]]

 [[ 0.9992863 -0.5987215]
  [-0.4572968 -0.811177 ]]]
Norm again [[[ 1.6407784 -1.2557573]
  [-0.4508798 -0.8140216]]

 [[ 0.9992863 -0.5987215]
  [-0.457296

ValueError: hi

In [15]:
[print_param(v) for v in tf.trainable_variables() ]

Name:  generator/dense/kernel:0
Shape:  (128, 2048)
[[-1.84785649e-02  2.19638906e-02  1.63171105e-02 ... -4.97036353e-02
   2.89887674e-02 -3.00705433e-05]
 [ 1.08730905e-02 -5.16189933e-02  1.73136778e-02 ...  3.70427258e-02
   5.29582798e-03 -4.71381061e-02]
 [-5.12372330e-03  1.89169459e-02 -2.38164943e-02 ...  4.30667736e-02
   3.25952880e-02  2.67593935e-03]
 ...
 [ 1.60910748e-02  1.30254664e-02  1.66432448e-02 ...  1.40613802e-02
   4.37478349e-03 -3.76919582e-02]
 [ 1.96285546e-03 -2.25370042e-02  2.85954066e-02 ...  3.60417254e-02
   1.12495199e-03  9.31483135e-03]
 [-3.74521464e-02 -5.67730889e-03  3.49582918e-02 ...  2.85275020e-02
   2.56133564e-02  3.49072628e-02]]
Name:  generator/dense/bias:0
Shape:  (2048,)
[0. 0. 0. ... 0. 0. 0.]
Name:  generator/batch_normalization/gamma:0
Shape:  (128,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [21]:
for v in tf.trainable_variables():
    if v.name == "generator/conv2d/kernel:0":
        x = sess.run(v)
    
print(x.shape)
print(np.round(x, 4))

(3, 3, 128, 128)
[[[[ 0.0063  0.0668 -0.0036 ...  0.0325  0.031  -0.014 ]
   [-0.0357  0.0247 -0.0519 ...  0.0236 -0.046  -0.0439]
   [-0.0586  0.0667  0.0048 ...  0.0506 -0.0011 -0.0602]
   ...
   [ 0.039   0.026   0.0249 ... -0.0318 -0.0659 -0.0619]
   [ 0.0586  0.0379  0.0611 ...  0.016  -0.006  -0.0391]
   [-0.0629  0.058   0.0303 ...  0.0295  0.0052 -0.0292]]

  [[-0.0569  0.0478  0.0292 ... -0.0126  0.0547 -0.0509]
   [-0.0274 -0.0698 -0.0083 ...  0.0274 -0.048  -0.0671]
   [-0.0486  0.0319 -0.0687 ...  0.0449  0.0442 -0.0662]
   ...
   [-0.0378  0.0239  0.0275 ... -0.0624  0.0005  0.0372]
   [-0.0452 -0.029  -0.0406 ...  0.0335 -0.0294  0.007 ]
   [ 0.0267 -0.0139 -0.0078 ...  0.0571 -0.0209  0.0242]]

  [[-0.0061  0.0247  0.0241 ... -0.0638 -0.0418  0.0468]
   [-0.072   0.043   0.0403 ...  0.0709  0.0452  0.0018]
   [-0.0204  0.0063 -0.0305 ...  0.023   0.0122 -0.0413]
   ...
   [-0.0116  0.0409  0.0438 ...  0.0154 -0.0251 -0.0524]
   [ 0.0025 -0.0592  0.0564 ...  0.0707 -0.046

In [ ]:
x = np.load("params/generator_dense_kernel:0.npy")
print(x.shape)
print(x[0,:3])

In [ ]:
x = np.load("params/generator_dense_kernel:0.npy")
print(np.sum(x, axis=0))

In [ ]:
# tf_arr = np.sum(first_dense_weight, axis=0)
# tf_arr = tf.convert_to_tensor(tf_arr, dtype=tf.float32)

sess = tf.InteractiveSession()
tf_arr = bn_input
print("BN Input", tf_arr[0,0,0,0])
print
bn = tf.contrib.layers.batch_norm(tf_arr, decay=0.9, # OK
                                    center=True, # OK
       f                              scale=True, #OK
                                    epsilon=1e-5,#OK
                                    zero_debias_moving_mean=False,
                                    is_training=True)
sess.run([tf.global_variables_initializer(),
          tf.local_variables_initializer()])
bn_eval = bn.eval()
bn_comparable = np.moveaxis(bn_eval, 3, 1)
print("output")
print(bn_comparable)
# [print_param(v) for v in tf.trainable_variables() ]
sess.close()

#0000, 1001 --> same val = same; x__x
#1100, 1110 --> same val = same  __x_ 1000

#

In [14]:
np.save("bn_input.npy", vec0[0])
np.save("bn_output.npy", vec1[0])
np.save("upsampled.npy", vec2[0])

In [13]:
for v in vec2:
    print(v[0,:2,:2,:2])
    
print(len(vec2))

[[[0.         0.21231355]
  [0.         0.21231355]]

 [[0.         0.21231355]
  [0.         0.21231355]]]
[[[0.         0.51633364]
  [0.         0.51633364]]

 [[0.         0.51633364]
  [0.         0.51633364]]]
[[[0.        1.3958745]
  [0.        1.3958745]]

 [[0.        1.3958745]
  [0.        1.3958745]]]
3


In [ ]:
BN INPUT: I FIXED THIS ALREADY [[[-0.36227444  0.13549557]
  [-0.47998536 -0.5358527 ]]

 [[-1.3878101   1.5702378 ]
  [-1.9097602   0.3231582 ]]]
BN RESULT: THIS SHOULD BE WORKING!!! [[[ 0.28651154 -0.47034988]
  [ 0.1410485  -1.3254068 ]]

 [[-0.9808096   1.3569971 ]
  [-1.6258174  -0.23133498]]]